In [4]:
!pip install haystack-ai accelerate sentence-transformers datasets

In [5]:
pip install cohere tiktoken

In [6]:
pip install pyarrow<15

/bin/bash: line 1: 15: No such file or directory


In [7]:
from haystack.telemetry import tutorial_running

tutorial_running(34)


In [8]:
# pip install wikipedia


^C


In [3]:
pip install PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.2 MB/s eta 0:00:00


In [27]:
import io
from datasets import load_dataset
from haystack import Document
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.readers import ExtractiveReader
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from datasets import Dataset
from haystack import Document, Pipeline
from PyPDF2 import PdfReader

from google.colab import files

uploaded = files.upload()

# Create a list of dictionaries with "content" and "meta" keys
# Here, we're setting the meta information as the title of the PDF file
# dataset = [{"content": uploaded, "meta": {"title": "your_pdf_file.pdf"}}]

# # Convert the list of dictionaries into a Dataset object
# documents = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]

# Extract content from PDF file
pdf_content = ""
for file_name, file_content in uploaded.items():
    with io.BytesIO(file_content) as f:
        pdf_reader = PdfReader(f)
        for page in pdf_reader.pages:
            pdf_content += page.extract_text()

# Create a list of dictionaries with "content" and "meta" keys
# Here, we're setting the meta information as the title of the PDF file
dataset = [{"content": pdf_content, "meta": {"title": "your_pdf_file.pdf"}}]

# Convert the list of dictionaries into a Dataset object
documents = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]


# Set up SentenceTransformersDocumentEmbedder
model = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
embedder = SentenceTransformersDocumentEmbedder(model=model)

# Set up DocumentWriter
document_store = InMemoryDocumentStore()
writer = DocumentWriter(document_store=document_store)

# Build the pipeline
indexing_pipeline = Pipeline()
indexing_pipeline.add_component(instance=embedder, name="embedder")
indexing_pipeline.add_component(instance=writer, name="writer")

indexing_pipeline.connect("embedder", "writer")

# Run the pipeline
indexing_pipeline.run(data={"documents": documents})



Saving MST_BF_Club_Christie_Bckgrnd.pdf to MST_BF_Club_Christie_Bckgrnd (5).pdf


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'writer': {'documents_written': 1}}

In [29]:
import pandas as pd


# Convert data to DataFrame
df = pd.DataFrame(documents)

# Display the DataFrame
df


,id,content,dataframe,blob,meta,score,embedding
0,b05dded78d1f48224e85169d68f2dbaac3d90e60780a57...,book & film club: Agatha ChristieBACKGROUND\t ...,None,None,{'title': 'your_pdf_file.pdf'},None,"[0.3049071729183197, 0.3375917971134186, -0.04..."


In [30]:

from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.readers import ExtractiveReader
from haystack.components.embedders import SentenceTransformersTextEmbedder


retriever = InMemoryEmbeddingRetriever(document_store=document_store)
reader = ExtractiveReader()
reader.warm_up()

extractive_qa_pipeline = Pipeline()

extractive_qa_pipeline.add_component(instance=SentenceTransformersTextEmbedder(model=model), name="embedder")
extractive_qa_pipeline.add_component(instance=retriever, name="retriever")
extractive_qa_pipeline.add_component(instance=reader, name="reader")

extractive_qa_pipeline.connect("embedder.embedding", "retriever.query_embedding")
extractive_qa_pipeline.connect("retriever.documents", "reader.documents")


In [31]:
query = "Who is Agatha?"
extractive_qa_pipeline.run(data={"embedder": {"text": query}, "retriever": {"top_k": 3},
                                   "reader": {"query": query, "top_k": 2}})


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'reader': {'answers': [ExtractedAnswer(query='Who is Agatha?', score=0.7167991995811462, data='Agatha ChristieMcEwan', document=Document(id=b05dded78d1f48224e85169d68f2dbaac3d90e60780a576e8134ce77a07501e6, content: 'book & film club: Agatha ChristieBACKGROUND	 	 page	1Background
   I do not know the whole Agatha. The ...', meta: {'title': 'your_pdf_file.pdf'}, score: 22.29453620328108), context=None, document_offset=ExtractedAnswer.Span(start=7738, end=7759), context_offset=None, meta={}),
   ExtractedAnswer(query='Who is Agatha?', score=0.5695516467094421, data='Colonel Archibald Christie', document=Document(id=b05dded78d1f48224e85169d68f2dbaac3d90e60780a576e8134ce77a07501e6, content: 'book & film club: Agatha ChristieBACKGROUND	 	 page	1Background
   I do not know the whole Agatha. The ...', meta: {'title': 'your_pdf_file.pdf'}, score: 22.29453620328108), context=None, document_offset=ExtractedAnswer.Span(start=3162, end=3188), context_offset=None, meta={}),
   ExtractedAnswer(query